In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.ensemble import IsolationForest

# Initialize counters for excluded days
excluded_days_due_to_points = 0
excluded_days_due_to_minutes = 0

# Initialize lists to collect outlier statistics and cleaned dataframes
outlier_stats = []
cleaned_dataframes = []

def detect_gaps(df, gap_threshold=30, pre_gap_buffer=60, post_gap_buffer=60, start_of_day=0, end_of_day=1440):
    df = df.sort_values(by='time_of_day').reset_index(drop=True)
    time_diffs = df['time_of_day'].diff().fillna(0)
    gap_indices = time_diffs[time_diffs > gap_threshold].index
    df['near_gap'] = False
    df['pre_gap'] = False
    df['post_gap'] = False

    # Flagging the start of the day if it starts late
    if df['time_of_day'].iloc[0] > start_of_day + post_gap_buffer:
        df.loc[0, 'post_gap'] = True
        buffer_indices = df[df['time_of_day'] <= df['time_of_day'].iloc[0] + post_gap_buffer].index
        df.loc[buffer_indices, 'post_gap'] = True

    for idx in gap_indices:
        immediate_range = range(max(0, idx - 1), min(len(df), idx + 2))
        df.loc[immediate_range, 'near_gap'] = True
        
        # Pre-gap buffer zone
        pre_gap_start = max(0, idx - pre_gap_buffer)
        df.loc[pre_gap_start:idx, 'pre_gap'] = True
        
        # Post-gap buffer zone
        post_gap_end = min(len(df), idx + post_gap_buffer)
        df.loc[idx:post_gap_end, 'post_gap'] = True

    # Flagging the end of the day if it ends early
    if df['time_of_day'].iloc[-1] < end_of_day - pre_gap_buffer:
        df.loc[len(df) - 1, 'pre_gap'] = True
        buffer_indices = df[df['time_of_day'] >= df['time_of_day'].iloc[-1] - pre_gap_buffer].index
        df.loc[buffer_indices, 'pre_gap'] = True

    return df



def process_date(df, specific_date, min_hours=3, min_points=300, gap_threshold=30, min_minutediff_threshold=0):
    global excluded_days_due_to_points, excluded_days_due_to_minutes
    df = df[df['date.1'] == specific_date].reset_index(drop=True)

    if df.shape[0] < min_points:
        print(f"Skipping {specific_date}: Not enough data points ({df.shape[0]})")
        excluded_days_due_to_points += 1
        return
    
    unique_minutes = df['time_of_day'] // 1
    if unique_minutes.nunique() < 180:
        print(f"Skipping {specific_date}: Not enough data minutes ({unique_minutes.nunique()})")
        excluded_days_due_to_minutes += 1
        return

    df = df[df['travel_time_minute'] > min_minutediff_threshold].reset_index(drop=True)
    df = detect_gaps(df, gap_threshold)
    
    # Apply LOWESS
    frac = 0.15  # Adjust this based on your data visualization and requirements
    it = 3      # Default is typically fine; adjust if necessary
    delta = 1.2 # Default; consider adjusting for large data
    lowess_smoothed = lowess(df['travel_time_minute'], df['time_of_day'], frac=frac, it=it, delta=delta)
    lowess_x, lowess_y = lowess_smoothed[:, 0], lowess_smoothed[:, 1]

    df['predicted'] = np.interp(df['time_of_day'], lowess_x, lowess_y)
    df['residual'] = df['travel_time_minute'] - df['predicted']
        # Calculate statistical metrics
    std = np.std(df['residual'])
    q75, q25 = np.percentile(df['residual'], [75 ,25])
    iqr = q75 - q25
    
    # Histogram of residuals
    plt.figure(figsize=(10, 6))
    plt.hist(df['residual'], bins=30, color='gray', alpha=0.6, label='Residuals Distribution')
    plt.axvline(x=3 * std, color='r', linestyle='--', linewidth=2, label='±3 STD')
    plt.axvline(x=-3 * std, color='r', linestyle='--', linewidth=2)
    plt.axvline(x=1.5 * iqr + q75, color='b', linestyle='--', linewidth=2, label='1.5 IQR upper')
    plt.axvline(x=q25 - 1.5 * iqr, color='b', linestyle='--', linewidth=2, label='1.5 IQR lower')
    plt.title(f'Residuals Distribution on {specific_date}')
    plt.xlabel('Residual Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
    slopes = np.gradient(lowess_y, lowess_x)
    df['high_slope'] = np.abs(slopes) > 0.5
    
    df['high_slope_near_gap'] = False
    df['high_slope_not_near_gap'] = False
    for idx in df.index[df['high_slope']]:
        range_indices = range(max(0, idx - 10), min(len(df), idx + 11))
        df.loc[range_indices, 'high_slope_near_gap'] = df.loc[range_indices, 'near_gap']
        df.loc[range_indices, 'high_slope_not_near_gap'] = ~df.loc[range_indices, 'near_gap']

    # Filter out the residuals that are significant outliers

    significant_residuals = (df['residual'].abs() > 3 * np.std(df['residual'])) | (df['residual'].abs() < -1 * np.std(df['residual']))
    df_filtered = df[~significant_residuals].reset_index(drop=True)
    
    # Parameters for Isolation Forest
    n_estimators = 100  # Number of base estimators in the ensemble
    max_samples = 'auto'  # Number of samples to draw from X to train each base estimator
    contamination = 0.01  # The proportion of outliers in the data set
    max_features = 1.0  # Number of features to draw from X to train each base estimator
    bootstrap = True  # If True, individual trees are fit on random subsets of the dataset sampled with replacement
    random_state = 42  # Ensures reproducibility of the results

    # Create the Isolation Forest instance
    iforest = IsolationForest(
        n_estimators=n_estimators,
        max_samples=max_samples,
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state,
        n_jobs=-1  # Utilize all processors for parallel processing
    )
    
    iforest_scores = iforest.fit_predict(df_filtered[['travel_time_minute']].values.reshape(-1, 1))
    df_filtered['iforest_score'] = iforest_scores
    
    df_filtered['ISO_slope_near'] = (df_filtered['iforest_score'] == -1) & df_filtered['high_slope_near_gap'] & (df_filtered['travel_time_minute']>30)
    df_filtered['ISO_slope_non_near'] = (df_filtered['iforest_score'] == -1) & df_filtered['high_slope_not_near_gap']  & (df_filtered['travel_time_minute']>30)
    df_filtered['ISO_slope_pre_gap'] = (df_filtered['iforest_score'] == -1) & df_filtered['pre_gap']  & (df_filtered['travel_time_minute']>30)
    df_filtered['ISO_slope_post_gap'] = (df_filtered['iforest_score'] == -1) & df_filtered['post_gap']  & (df_filtered['travel_time_minute']>30)
    df_filtered['ISO_slope_near_gap'] = (df_filtered['iforest_score'] == -1) & df_filtered['near_gap']  & (df_filtered['travel_time_minute']>30)
    
    outlier_stats.append({
        'date': specific_date,
        'total_points': len(df_filtered),
        'lowess_outliers': np.sum(significant_residuals),
        'ISO_slope_near': np.sum(df_filtered['ISO_slope_near']),
        'ISO_slope_non_near': np.sum(df_filtered['ISO_slope_non_near'])
    })
    
    # Combine all gap-related conditions into a single Boolean Series
    near_gap_conditions = (
        df_filtered['ISO_slope_near'] | 
        df_filtered['ISO_slope_non_near'] | 
        df_filtered['ISO_slope_pre_gap'] | 
        df_filtered['ISO_slope_post_gap'] | 
        df_filtered['ISO_slope_near_gap']
    )

    # Plot the results
    plt.figure(figsize=(15, 7))
    plt.scatter(df['time_of_day'], df['travel_time_minute'], alpha=0.3, label='Data Points', c='grey')
    plt.plot(lowess_x, lowess_y, color='blue', label='LOWESS Fit', linewidth=2)
    plt.scatter(df['time_of_day'][significant_residuals], df['travel_time_minute'][significant_residuals], color='red', edgecolor='k', s=50, label='LOWESS Outliers')
    plt.scatter(
        df_filtered['time_of_day'][near_gap_conditions], 
        df_filtered['travel_time_minute'][near_gap_conditions],
        color='purple',  # Choose a color that signifies all 'near gap' points
        edgecolor='k', 
        s=50, 
        label='ISO-Near Gap'  # Unified label
    )
    
    plt.xlabel('Time of Day (minutes from 0 to 1440)', fontsize=14)
    plt.ylabel('Travel Time (minutes)', fontsize=14)
    plt.xticks(range(0, 1441, 60), fontsize=12)
    plt.yticks(fontsize=12)
    plt.title(f'Travel Time vs Time of Day on {specific_date}', fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    condition = (
        df_filtered['ISO_slope_near'] | df_filtered['ISO_slope_non_near'] |
        df_filtered['ISO_slope_pre_gap'] | df_filtered['ISO_slope_post_gap'] |
        df_filtered['ISO_slope_near_gap']
    )
    df_filtered = df_filtered[~condition].reset_index(drop=True)
    
    cleaned_dataframes.append(df_filtered)    

def save_cleaned_data(filename='your_dataset.csv'):
    # Concatenate all cleaned dataframes
    all_cleaned_data = pd.concat(cleaned_dataframes)
    # Save to CSV
    all_cleaned_data.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Assuming 'df' is your DataFrame and is already loaded
unique_dates = df['date.1'].unique()
for specific_date in unique_dates:
    process_date(df, specific_date, min_hours=3, min_points=300, gap_threshold=30)

save_cleaned_data('your_dataset.csv')  # Save the cleaned data to a CSV file
